# Create standard dataset

## Import library

In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm
import os

## 데이터 전처리 과정

In [2]:
game_rule = 'Standard'
base_path = 'dataset'

output_path = os.path.join(os.path.basename(base_path), 'dataset_1515' )
os.makedirs(output_path, exist_ok=True)

In [3]:
# 경로에 있는 모든 psq파일을 file_list에 저장
file_list = glob(os.path.join(base_path, '%s*/*.psq' % (game_rule, )))

In [4]:
for index, file_path in enumerate(tqdm(file_list)):
    with open(file_path, 'r') as f:
        lines = f.read().splitlines()

    w, h = lines[0].split(' ')[1].strip(',').split('x')
    w, h = int(w), int(h)

    lines = lines[1:]

    inputs, outputs = [], []
    board = np.zeros([h, w], dtype=np.int8)

    for i, line in enumerate(lines):
        if ',' not in line:
            break
        x, y, t = np.array(line.split(','), np.int8)

        if i % 2 == 0:
            player = 1
        else:
            player = 2

        input1 = board.copy().astype(np.int8)
        input1[(input1 != player) & (input1 != 0)] = -1
        input1[(input1 == player) & (input1 != 0)] = 1

        output = np.zeros([h, w], dtype=np.int8)
        output[y-1, x-1] = 1

        for k in range(4):
            input_rot = np.rot90(input1, k=k)
            output_rot = np.rot90(output, k=k)

            inputs.append(input_rot)
            outputs.append(output_rot)

            inputs.append(np.fliplr(input_rot))
            outputs.append(np.fliplr(output_rot))

            inputs.append(np.flipud(input_rot))
            outputs.append(np.flipud(output_rot))

        board[y-1, x-1] = player
    np.savez_compressed(os.path.join(output_path, '%s.npz' % (str(index).zfill(5))), inputs=inputs, outputs=outputs)

100%|██████████| 2496/2496 [00:30<00:00, 80.78it/s] 
